In [1]:
!jt -t chesterish -fs 12 -nfs 12 -tfs 11 -T -ofs 10 -cellw 1600 -f dejavu

zsh:1: command not found: jt


In [1]:
import re

In [2]:
class thai_match_pattern():
    def __init__(self, start_regex, vowel_regex, end_regex, sound_regex):
        self.start_regex = start_regex
        self.vowel_regex = vowel_regex
        self.end_regex = end_regex
        self.sound_regex = sound_regex
        
    def match_start(self, orig_word):        
        matches = re.findall(self.start_regex, orig_word, re.DOTALL)
        return matches
    

In [3]:
#Reference https://www.geeksforgeeks.org/longest-common-substring-dp-29/
# Returns length of longest common  
# substring of X[0..m-1] and Y[0..n-1]  
def LCSubStr(X, Y, m, n): 
      
    # Create a table to store lengths of 
    # longest common suffixes of substrings.  
    # Note that LCSuff[i][j] contains the  
    # length of longest common suffix of  
    # X[0...i-1] and Y[0...j-1]. The first 
    # row and first column entries have no 
    # logical meaning, they are used only 
    # for simplicity of the program. 
      
    # LCSuff is the table with zero  
    # value initially in each cell 
    LCSuff = [[0 for k in range(n+1)] for l in range(m+1)] 
      
    # To store the length of  
    # longest common substring 
    result = 0 
  
    # Following steps to build 
    # LCSuff[m+1][n+1] in bottom up fashion 
    for i in range(m + 1): 
        for j in range(n + 1): 
            if (i == 0 or j == 0): 
                LCSuff[i][j] = 0
            elif (X[i-1] == Y[j-1]): 
                LCSuff[i][j] = LCSuff[i-1][j-1] + 1
                result = max(result, LCSuff[i][j]) 
            else: 
                LCSuff[i][j] = 0
    return result 

In [4]:
import re
class thai_subwords():
    """Design to take input as single Thai token or word and extract start, vowel, end and sound of said token
    For additional data regarding different Thai words types and pronunciations, 
    consult:https://sites.google.com/site/phasathaionline/hnwy-kar-reiyn-ru2/phyaychnatn"""
    def __init__(self, orig_word, debug=False):
        self.orig_word = orig_word #Try not to change this value
        self._start = ''
        self._vowel = ''
        self._end = ''
        self._sound = ''
        self._double_start_sound = False #used to flag with that has two starting alphabets with sound like 'ตลาด = ตะ ลาด'
        self._high_lead_low = False #case อักษรสูงนำอักษรต่ำ
        self.debug = debug
        
        #Need to deal with negative lookahead case for word like "เกรียน" to correctly extract กร
        self.vowels_form = ['ะ', ' ั' ' ็', 'า', ' ิ' , ' ่' , " ่" , ' ํ', ' ุ', ' ู', 'เ', 'ใ', 'ไ', 'โ', 'อ', 'ย', 'ว', 'ฤ', 'ฤๅ', 'ฦ', 'ฦๅ']
        combine_vowel_regex_1 = '(?!เ.{1,4}ี.*ย)(?!เ.{1,4}ี.*ยะ)(?!เ.{1,4}ื.*อ)(?!เ.{1,4}ื.*อะ)(?!เ.{1,4}.*อะ)(?!เ.{1,4}.*อ)(?!เ.{1,4}.*าะ)'
        combine_vowel_regex_2 = '(?!เ.{1,4}.*า)(?!แ.{1,4}.*า)(?!.{1,4}ัวะ)(?!โ.{1,4}.*ะ)'
        combine_vowel_regex = combine_vowel_regex_1 + combine_vowel_regex_2
        self.start_regex_1 = ['(.{1,4})ะ','(.{1,4})า', '(.{1,4})ิ.*', '(.{1,4})ี.*', '(.{1,4})ึ', '(.{1,4})ื.*', '(.{1,4})ุ.*', '(.{1,4})ู.*', 'เ(.{1,4})็', 
                            'เ(.{1,4}).*', '(.{1,4}).*อ','แ(.{1,4}).*', '(.{1,4})ั.*', 'โ(.{1,4}).*', '(.{1,4}).*ำ', 'ใ(.{1,4}).*', 'ไ(.{1,4})']
        self.start_regex_1  = [combine_vowel_regex+regex for regex in self.start_regex_1]
        self.start_regex_2 = ['เ(.{1,4})ี.*ยะ', 'เ(.{1,4})ี.*ย', 'เ(.{1,4})ื.*อะ', 'เ(.{1,4})ื.*อ', 'เ(.{1,4}).*อ', 'เ(.{1,4}).*อะ',
                              'เ(.{1,4}).*า',
                              'เ(.{1,4}).*าะ','แ(.{1,4}).*ะ','(.{1,4})ัวะ','โ(.{1,4}).*ะ', 'เ(.{1,4})็', 'เ(.{1,4})ิ.*']
        self.end_regex_1 = ['\[REP\]า(.{1,4})', '\[REP\]ิ(.{1,4})', '\[REP\]ี(.{1,4})', '\[REP\]ึ(.{1,4})', '\[REP\]ือ(.{1,4})', 
                            '\[REP\]ุ(.{1,4})', '\[REP\]ู(.{1,4})', 'เ\[REP\]็(.{1,4})', 'เ\[REP\](.{1,4})', '\[REP\]อ(.{1,4})',
                            'แ\[REP\](.{1,4})', '\[REP\]ั(.{1,4})', 'โ\[REP\](.{1,4})','\[REP\]ำ(.{1,4})', 'ใ\[REP\](.{1,4})', 
                            'ไ\[REP\](.{1,4})']
        self.end_regex_2 = ['เ\[REP\]ีย(.{1,4})', 'เ\[REP\]ือ(.{1,4})', 'เ\[REP\]อ(.{1,4})', 'เ\[REP\]อะ(.{1,4})','เ\[REP\]า(.{1,4})']
        
        self.start_regex  = self.start_regex_1 + self.start_regex_2 
        self.end_regex  = self.end_regex_1 + self.end_regex_2 
        #self.vowel_regex = vowel_regex
        #self.end_regex = end_regex
        self.sound_regex = ['(่)', '(้)', '.(๊)','()๋']
        
        #initialize the special character cases
        #คำควบแท้
        self.two_char_combine = ['กร','กล', 'กว', 'คร', 'ขร', 'คล', 'ขล','คว', 'ขว', 'ตร', 'ปร', 'ปล', 'พร', 'พล' ,'ผล',
                   'บร','บล','ดร','ฟร','ฟล','ทร','จร','ซร','ปร','สร']
        #คำนำ
        self.lead_char_nosound = ['อย','หง','หญ','หน','หม','หย', 'หร','หล','หว']
        #อักษรสูงนำอักษรต่ำ
        self._lead_char_high_low = ['ขน', 'ขม','สม','สย','สน','ขย','ฝร','ถล','ผว','ตน','จม','ตล',]
        #อักษรสูงนำอักษรกลาง
        self._high_char_high_medium = ['ผท','ผด','ผก','ผอ','ผช']
        
        
    # using property decorator 
    @property
    def start(self): 
        #print("getter method called") 
        temp_start = self.match_pattern(self.orig_word, self.start_regex)
        if len(temp_start)==1: return temp_start
        else: 
            return self.check_double_start_alphabets(temp_start)
        #return self.match_pattern(self.orig_word, self.start_regex)
    
    @property
    def sound(self):
        if self.match_pattern(self.orig_word, self.sound_regex) is None: return ''
        return self.match_pattern(self.orig_word, self.sound_regex)
    
    @property
    def vowel(self):
        return self.orig_word.replace(self.sound,'').replace(self.start,'[REP]',1) #[REP] will be used for conversion to Lu/Ru
#     def vowel(self):
#         return self.orig_word.replace(self.sound,'').replace(self.end,'').replace(self.start,'[REP]',1) #[REP] will be used for conversion to Lu/Ru
    
    @property #might not need this one in final application as we need vowel
    def end(self):
        if self.match_pattern(self.vowel, self.end_regex) not in self.vowels_form: 
            return self.match_pattern(self.vowel, self.end_regex)
        else:
            return None

#     # a setter function 
#     @start.setter 
#     def start(self): 
#         print("setter method called") 
#         self._start = match_pattern(self.start_regex)
    
    def match_pattern(self, text, regex_pattern):
        output_start = None
        if self.debug: print(text)
        for pattern in regex_pattern:
            matches = re.search(pattern, text, re.DOTALL)
            
            if matches is not None and self.debug: print(pattern, matches.group(1))
            if matches is not None: output_start = matches.group(1)
        return output_start
    
    def check_double_start_alphabets(self, text):
        '''Use to check in case starting alphabet is longer than one, 
        ref:https://sites.google.com/site/phasathaionline/hnwy-kar-reiyn-ru2/phyaychnatn'''
        #Function that check word in case like ขนม and else 
        
        two_char = self.lead_char_nosound + self.two_char_combine
        
        max_lcs = 0
        if text in two_char: return text
        else:
            for char in two_char:
                if LCSubStr(text, char, len(text), len(char)) > max_lcs: 
                    max_lcs = LCSubStr(text, char, len(text), len(char))
                    max_lcs_char = char
            return max_lcs_char
        
        return None
    
    def check_double_start_sound(self, text):
        '''Use to address the case where start alphabets are double and need to split to reflect actual pronunciation 
        ref:https://sites.google.com/site/phasathaionline/hnwy-kar-reiyn-ru2/phyaychnatn'''
        #Function that check word in case like ขนม and else 
        
        
        return None
    
    def remove_karan(self, regex_pattern):
        return None
        #Write specific function for removing ์

In [5]:
subtest = thai_subwords('ปลาด', debug=False)
print(subtest.start)
print(subtest.vowel)
print(subtest.end)
subtest.sound

#subtest = thai_subwords()

ปล
[REP]าด
ด


''

In [6]:
from pythainlp.tokenize import syllable_tokenize #use syllable level as it should be easier for Lu conversion
class Lu_translators():
    def __init__(self, sentence, syllable_tokenizer=syllable_tokenize):
        self.orig_sen = sentence
        self.orig_tokens = syllable_tokenizer(self.orig_sen)
        self.thai_to_lu_dict = {}
        #get subwords
        self.sub_words = [thai_subwords(token) for token in self.orig_tokens if token != ' ']
    
    def thai_to_lu(self, sub_words):
        '''Does not use self.subwords in case we want to apply to other objects
        input = list of sub_words object'''
        
        thai_to_lu_dict = {}
        for token in sub_words:
            #print(token.orig_word)
            ending = token.end if token.end is not None else ''
            start_rep = 'ซ' if token.start == 'ล' or token.start == 'ร' else 'ล'   #start replacement, can be ล or ซ
            if '[REP]ู' in token.vowel:
                lu = token.vowel.replace('[REP]', start_rep) + token.start + 'ี' + ending
            elif '[REP]ุ' in token.vowel:
                lu = token.vowel.replace('[REP]', start_rep) + token.start + 'ิ' + ending
            else:
                lu = token.vowel.replace('[REP]', start_rep) + token.start + 'ู' + ending
            thai_to_lu_dict[token.orig_word] = lu
        return thai_to_lu_dict
    
    def check_longshort_sound():
        return None
        #Need to write function to check เสียงสั้นเสียงยาว
    

In [14]:
test_lu = Lu_translators("ไปไหนดี")
print(test_lu.orig_tokens)
thai_to_lu_dict = test_lu.thai_to_lu(test_lu.sub_words)
# for sub in test_lu.sub_words:
#     print(sub.orig_word)
#     print(sub.start, sub.vowel)
print(thai_to_lu_dict)

['ไป', 'ไหน', 'ดี']
{'ไป': 'ไลปู', 'ไหน': 'ไลหนู', 'ดี': 'ลีดู'}


In [11]:
test_lu = Lu_translators("สวย")
print(test_lu.orig_tokens)
thai_to_lu_dict = test_lu.thai_to_lu(test_lu.sub_words)
# for sub in test_lu.sub_words:
#     print(sub.orig_word)
#     print(sub.start, sub.vowel)
print(thai_to_lu_dict)

['สวย']


TypeError: object of type 'NoneType' has no len()

In [9]:
test_lu = Lu_translators("ตดดังมาก")
print(test_lu.orig_tokens)
thai_to_lu_dict = test_lu.thai_to_lu(test_lu.sub_words)
# for sub in test_lu.sub_words:
#     print(sub.orig_word)
#     print(sub.start, sub.vowel)
print(thai_to_lu_dict)

['ตด', 'ดัง', 'มาก']


TypeError: object of type 'NoneType' has no len()